In [1]:
from typing import Tuple

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from tick.hawkes import SimuHawkes, HawkesKernelSumExp, HawkesKernelExp
from tick.hawkes import HawkesExpKern

In [4]:
class UnivariateHawkes():
    """ """
    
    def __init__(self,
                baseline:float = None, 
                alpha: float = None,
                beta: float = None,
                total_run_time: int = 100,
                time_step: float = 0.01,
                max_iter: int = 10000,
                tol: float = 1e-10,
                min_beta: float = 0,
                max_beta: float = 30,
                verbose: bool = False,
                seed: int = 1500
                ):
        self.number_of_nodes = 1
        self.baseline = baseline
        self.alpha = alpha
        self.beta = beta
        self.total_run_time = total_run_time
        self.time_step = time_step
        self.max_iter = max_iter
        self.tol= tol 
        self.min_beta = min_beta
        self.max_beta = max_beta
        self.verbose = verbose
        self.seed = seed 
        
        
        
    def fit(self, events):
            
        beta_candidates = np.linspace(0, 30, 100)
        scores = []
        hawkes_models = {}
        for i, beta in enumerate(beta_candidates):
            hawkes_learner = HawkesExpKern(beta, 
                                           verbose=self.verbose, 
                                           max_iter=self.max_iter,
                                           tol=self.tol)
            hawkes_learner.fit(events)
            hawkes_models[i] = hawkes_learner
            scores.append(hawkes_learner.score() )
    
        beta_scores_df = pd.DataFrame({'score': scores, 
                               'decay':beta_candidates}).sort_values(by = 'score', ascending = False)
        best_hawkes_model_index = beta_scores_df.index[0]
        best_hawkes_model = hawkes_models[best_hawkes_model_index]
        self.baseline = best_hawkes_model.baseline[0]
        self.alpha = best_hawkes_model.adjacency[0][0]
        self.beta = best_hawkes_model.decays
        
        
            
    
    def simulate(self)-> Tuple[np.ndarray, np.ndarray, np.ndarray]:
         
        hawkes = SimuHawkes(n_nodes=self.number_of_nodes, 
                            end_time=self.total_run_time, 
                            verbose=self.verbose, 
                            seed=self.seed)
        kernel = HawkesKernelExp(intensity=self.alpha, 
                                    decay= self.beta)
        hawkes.set_kernel(0, 0, kernel)
        hawkes.set_baseline(0, self.baseline)
        hawkes.track_intensity(self.time_step)
        hawkes.simulate()
        
        return hawkes.timestamps, hawkes.tracked_intensity, hawkes.intensity_tracked_times
        
        
    
    
    
    def get_conditional_intensity(self,
                                  events: np.ndarray, 
                                  current_step: float) -> float:

        return self.baseline + np.sum(self.alpha*self.beta * np.exp(-self.beta * (current_step - events)))            
    
    
    def get_next_arrival_time(self,
                                  events: np.ndarray, 
                                  current_step: float)-> float:
        
        predicted_intensity = self.get_conditional_intensity(events, current_step)
        
        return current_step + np.random.exponential(1 / predicted_intensity)
    

In [5]:
SEED = 1398
total_run_time = 100
time_step = 0.01
true_alpha = 0.5 # self-excitation intensity
true_beta = 3 # decay
true_baseline = 1 # mu


In [6]:
hawkes_model = UnivariateHawkes(baseline        = true_baseline, 
                                alpha           = true_alpha,
                                beta            = true_beta,
                                total_run_time  = total_run_time,
                                time_step       = time_step,
                                max_iter        = 10000,
                                tol             = 1e-10,
                                min_beta        = 0,
                                max_beta        = 30,
                                verbose         = False,
                                seed            = SEED)

In [7]:
simulated_info = hawkes_model.simulate()
simulated_events, simulated_tracked_intensity, simulated_intensity_tracked_times = simulated_info

hawkes_model.fit(simulated_events)

Step equals 0... at 0


In [8]:
true_baseline, true_alpha, true_beta

(1, 0.5, 3)

In [9]:
hawkes_model.baseline, hawkes_model.alpha, hawkes_model.beta

(1.0766456880520174, 0.46659430920350237, 3.3333333333333335)

In [10]:
hawkes_model.get_next_arrival_time(events=simulated_events[0], 
                                   current_step=simulated_events[0][-1])

100.10642847814678

In [11]:
import attrs
@attrs.define
class UnivariateHawkesModel():
    
    """A wrappper class leveraging tick's implementation of univariate Hawkes process
    
    For more detaisl, consult:
    https://x-datainitiative.github.io/tick/modules/generated/tick.hawkes.HawkesExpKern.html#tick.hawkes.HawkesExpKern
    
    """
    
    number_of_nodes: int = 1
    baseline:float = None 
    alpha: float = None
    beta: float = None
    max_iter: int = 10000
    tol: float = 1e-10
    min_beta: float = 0
    max_beta: float = 30
    verbose: bool = False
    seed: int = 1500
        
        
    def fit(self, events: np.ndarray):
        """Fits the Hawkes parameters to the input events realization  """    
        beta_candidates = np.linspace(self.min_beta, self.max_beta, 100)
        scores = []
        hawkes_models = {}
        for i, beta in enumerate(beta_candidates):
            hawkes_learner = HawkesExpKern(beta, 
                                           verbose=self.verbose, 
                                           max_iter=self.max_iter,
                                           tol=self.tol)
            hawkes_learner.fit(events)
            hawkes_models[i] = hawkes_learner
            scores.append(hawkes_learner.score() )
    
        beta_scores_df = pd.DataFrame({'score': scores, 
                               'decay':beta_candidates}).sort_values(by = 'score', ascending = False)
        best_hawkes_model_index = beta_scores_df.index[0]
        best_hawkes_model = hawkes_models[best_hawkes_model_index]
        self.baseline = best_hawkes_model.baseline[0]
        self.alpha = best_hawkes_model.adjacency[0][0]
        self.beta = best_hawkes_model.decays
        
        
            
    
    def simulate(self,
                total_run_time: int = 100,
                time_step: float = 0.01)-> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """ Simulates a realization of events from existing model parameters. """ 
        hawkes = SimuHawkes(n_nodes=self.number_of_nodes, 
                            end_time=total_run_time, 
                            verbose=self.verbose, 
                            seed=self.seed)
        kernel = HawkesKernelExp(intensity=self.alpha, 
                                    decay= self.beta)
        hawkes.set_kernel(0, 0, kernel)
        hawkes.set_baseline(0, self.baseline)
        hawkes.track_intensity(time_step)
        hawkes.simulate()
        
        return hawkes.timestamps, hawkes.tracked_intensity, hawkes.intensity_tracked_times
        
    def get_conditional_intensity(self,
                                  events: np.ndarray, 
                                  current_step: float) -> float:
        """Computes the conditional intensity given a realization of events """

        return self.baseline + np.sum(self.alpha*self.beta * np.exp(-self.beta * (current_step - events)))            
    
    
    def get_next_arrival_time(self,
                                  events: np.ndarray, 
                                  current_step: float)-> float:
        """Computes the  expected next arrival time given an event realization and the current step"""
        
        predicted_intensity = self.get_conditional_intensity(events, current_step)
        
        return current_step + np.random.exponential(1 / predicted_intensity)
    

In [12]:
hawkes_model = UnivariateHawkesModel(baseline        = true_baseline, 
                                        alpha           = true_alpha,
                                        beta            = true_beta,
                                        max_iter        = 10000,
                                        tol             = 1e-10,
                                        min_beta        = 0,
                                        max_beta        = 30,
                                        verbose         = False,
                                        seed            = SEED)

In [13]:
simulated_info = hawkes_model.simulate(total_run_time,time_step)
simulated_events, simulated_tracked_intensity, simulated_intensity_tracked_times = simulated_info

hawkes_model.fit(simulated_events)

Step equals 0... at 0


In [14]:
true_baseline, true_alpha, true_beta

(1, 0.5, 3)

In [15]:
hawkes_model.baseline, hawkes_model.alpha, hawkes_model.beta

(1.0766456880520174, 0.46659430920350237, 3.3333333333333335)

In [16]:
hawkes_model.get_next_arrival_time(events=simulated_events[0], 
                                   current_step=simulated_events[0][-1])

100.10642847814678